<a href="https://colab.research.google.com/github/psaboia/CV2_project/blob/main/clutterizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Nathan Vance's code with Jeremy Speth's and Priscila's changes.  
import os
import sys
from os.path import isfile, join
from os import listdir
import math
import random
import json
from imutils import paths
from tqdm import tqdm
import numpy as np
import cv2
from skimage import transform
from shapely.geometry import Polygon, LineString
from scipy.ndimage.interpolation import rotate
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow

def sample_background(img_path):  
  file_names = list(paths.list_images(img_path))
  file_name = np.random.choice(file_names)
  img = cv2.imread(file_name)  
  return img

def sample_objects(class_indivs, categories_dict, min_objects_per_image, max_objects_per_image):
  num_objects = np.random.randint(min_objects_per_image,max_objects_per_image)
  category_ids = list(categories_dict.keys())
  obj_ids = np.random.choice(category_ids, num_objects, replace=False)
  obj_list = [np.random.choice(class_indivs[obj_id]) for obj_id in obj_ids]
  return obj_list, obj_ids    

def load_coco_instances(json_path):

  # load JSON
  with open(json_path, 'r') as infile: data = json.load(infile)
  # dictionary for images
  images_dict = {i['id']: i['file_name'] for i in data['images']}
  # dictionary for categories
  categories_dict = {i['id']: i['name'] for i in data['categories']}
  
  # list of annotations by category_id
  anno_list = {}
  for anno in data['annotations']:
      category_id = anno['category_id']
      if category_id not in anno_list.keys():
          anno_list[category_id] = [anno]
      else:
          anno_list[category_id].append(anno)

  return anno_list, images_dict, categories_dict

def plot_img(img, figsize=(5, 5)):
  plt.figure(figsize=figsize)
  plt.imshow(img)
  # turns off axes
  plt.axis("off")
  # gets rid of white border
  plt.axis("tight")
  # square up the image instead of filling the "figure" space
  plt.axis("image")
  plt.show()    
  
  #im_v = cv2.vconcat([img1, img1]) 


def get_annotation(mask, ann_id, image_id, category_id): 

  elem = {}
  if np.sum(mask) > 0:
    # for labeled mask
    # label_temp[comb_label == obj_id+1] = (obj_id +1)*25

    ## Writing json elements:      
    # <ann_id> element
    elem['id'] = ann_id

    # <image_id> element
    elem['image_id'] = image_id

    # <category_id> element
    elem['category_id'] = category_id

    # - find segmented objects by using cv2.findContours 
    contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # - find the segmentation point list, list of boxes and total area
    #   Note: If the object is occluded, it is possible that the object has  
    #   been broken in non connected parts.
    list_seg = []
    list_pts = []
    list_boxes = []
    area = 0
    for c in contours:
      # - list of contours in the COCO format annotation for segmentation
      points = np.array([[float(point[0][0]), float(point[0][1])] for point in c])
      seg_points = np.hstack(points).tolist()
      if len(seg_points) % 2 == 0 and len(seg_points) >= 6:
        list_seg.append(seg_points)
        area += cv2.contourArea(c)

      # - list of boxes
      (x, y, w, h) = cv2.boundingRect(c)
      list_boxes.append([x,y, x+w,y+h])        

    # <segmentation> element
    elem['segmentation'] = list_seg

    # <area> element
    elem['area'] = int(area)

    # <bbox> element
    elem['bbox_mode'] = 1 # BoxMode.XYWH_ABS: 1 / BoxMode.XYXY_ABS: 0 
    boxes = np.asarray(list_boxes)
    x1, y1 = np.min(boxes, axis=0)[:2]
    x2, y2 = np.max(boxes, axis=0)[2:]     
    elem['bbox'] = [float(z) for z in [x1,y1,abs(x2-x1),abs(y2-y1)]]

    # <iscrowd> element
    elem['iscrowd'] = 0

    # - show contours for debug 
    # im = mask.copy()
    # im = cv2.drawContours(im, contours, -1, (0,0,255), 10)        
    # im = cv2.rectangle(im,(x1,y1),(x2,y2),(255,0,0),5)
    # cv2_imshow(im)
    # plot_img(im, figsize=(10, 10))

  return elem

def add_annotations(annos, ann_id, image_id, obj_ids, obj_list, comb_label, mask):
  for obj_id, obj in zip(obj_ids, obj_list):

      # binary mask for the object
      mask = (comb_label == (obj_id+1)).astype(np.uint8)
      
      # JSON annotation element for the object
      
      anno = get_annotation(mask, ann_id+1, image_id, obj['category_id'])
      
      if anno: 
        ann_id += 1
        # add annotation
        annos.append(anno)

  return annos, ann_id

def get_categories(categories_dict):
  categories = []
  for c in categories_dict:
    elem_cat = {}
    elem_cat['id'] = c
    elem_cat['name'] = categories_dict[c] 
    categories.append(elem_cat)
  return categories

def get_image_dict(im_file_name, comb_width, comb_height):
  im_dict = {}
  im_dict['id'] = image_id
  im_dict['file_name'] = im_file_name
  im_dict['height'] = comb_height
  im_dict['width'] = comb_width
  return im_dict


def add_obj_to_img(obj_img, obj_id, comb_image, comb_label):
  comb_height, comb_width = comb_label.shape
  img_height, img_width, _ = obj_img.shape
  segm = np.array(obj['segmentation']).reshape(-1,2)

  # plot_img(obj_img, figsize=(3, 3)) #cv2_imshow(obj_img)
    
  ### Rotate segmentation
  rotation = np.random.uniform(0, 2*np.pi)
  tform = transform.SimilarityTransform(rotation=rotation)
  tform_mat = tform.params
  segm_homog = np.append(segm, np.ones(len(segm)).reshape(-1, 1), 1)
  segm_tformed = segm_homog.dot(tform_mat.T)[:,:2]

  ### Shift it back to [0,0]
  segm_tformed -= np.min(segm_tformed, 0)    

  ### Pad image to scene size
  pad_image = np.zeros((comb_height, comb_width, 3), dtype=np.uint8)
  pad_image[:img_height, :img_width] = obj_img    
  #cv2.imwrite(f'temp/{i}_{obj_id}_pad.jpg', pad_image)    

  x1,y1 = segm_tformed.min(0)
  x2,y2 = segm_tformed.max(0)
  width = x2-x1
  height = y2-y1
  scale = np.random.uniform(0.20, 0.6)
  translation = (np.random.randint(0, comb_width-(width*scale)), np.random.randint(0, comb_height-(height*scale)))
  #translation = (np.random.randint(0, img_width-(width*scale)), np.random.randint(0, img_height-(height*scale)))


  tform = transform.SimilarityTransform(scale=scale, translation=translation)
  tform_mat = tform.params
  segm_homog = np.append(segm_tformed, np.ones(len(segm_tformed)).reshape(-1, 1), 1)
  segm_tformed = segm_homog.dot(tform_mat.T)[:,:2]


  tform = transform.SimilarityTransform()
  tform.estimate(segm, segm_tformed)
  frame_tformed = transform.warp(pad_image, tform.inverse, preserve_range=True)
  frame_tformed = frame_tformed.astype(np.uint8)
  segm_tformed = segm_tformed.astype(np.int32)   
  
  # plot_img(frame_tformed, figsize=(3, 3))

  segm_tformed[:,0] = np.clip(segm_tformed[:,0], 0, comb_width)
  segm_tformed[:,1] = np.clip(segm_tformed[:,1], 0, comb_height)
  segm_tformed = segm_tformed.reshape(-1,1,2).astype(np.int32)

  mask = np.zeros(frame_tformed.shape[:3], dtype=np.uint8)     


  # fill the ROI into the mask
  #for point in segm_tformed:
  #    mask = cv2.circle(mask, tuple(point[0]), 5, (0,255,0), -1)
  cv2.fillPoly(mask, [segm_tformed], (1,1,1))

  # The mask image
  #cv2.imwrite(f'temp/{i}_{obj_id}_mask.jpg', mask*255)    
  #plot_img( mask*255, figsize=(3, 3))

  mask_gray = mask[:,:,0]
  #contours, hierarchy = cv2.findContours(mask_gray, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

  ### applying the mask to original image
  masked_image = frame_tformed * mask

  comb_image[mask==1] = frame_tformed[mask==1]
  #comb_label[mask_gray==1] = obj_id*25
  comb_label[mask_gray==1] = obj_id+1

  #cv2.imwrite(f'temp/{i}_{obj_id}_masked.jpg', masked_image)
  #cv2.imwrite(f'temp/{i}_{obj_id}_0.jpg', obj_image)
  #cv2.imwrite(f'temp/{i}_{obj_id}_comb.jpg', comb_image)
  #cv2.imwrite(f'temp/{i}_{obj_id}_label.jpg', comb_label)
  #print()    

  #plot_img( masked_image, figsize=(3, 3))
  #plot_img( obj_img, figsize=(3, 3))
  #plot_img( comb_image, figsize=(3, 3))
  #plot_img( comb_label, figsize=(3, 3))

  # cv2_imshow(comb_image)
  return comb_image, comb_label, mask

In [ ]:
from os.path import isfile, isdir, join
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Individual objects dataset path
individual_objects_path = "/content/drive/MyDrive/CSE/Spring21/cv2/dataset/prix_objects_v1/train"
individual_objects_json = join(individual_objects_path, "annotations.json")

# Background images path
img_bg_path = "/content/drive/MyDrive/CSE/Spring21/cv2/dataset/background"

# output path
outdir = "/content/drive/MyDrive/CSE/Spring21/cv2/dataset/SyntheticClutteredObjects_v3"
if not os.path.isdir(outdir): os.makedirs(outdir)

In [ ]:
task_number = 50

# seting up augmentation
num_imgs = 10     # number of synthetic images to be generated
min_obj = 9         # min number of objects per image
max_obj = 10        # max number of objects per image
synt_width = 3024   # width of the synthetic image
synt_height = 3024  # height of the synthetic image

In [ ]:

# Read a file containing JSON object on COCO format whith the individual object instances
anno_list, images_dict, categories_dict = load_coco_instances(individual_objects_json)  

start = task_number
end = start + num_imgs
size = (synt_height, synt_width)

data = {}
images = []
annos = []
ann_id = 0
for image_id in tqdm(range(start, end)):
  print(image_id)
  # Choose sampling instances to form a cluttered synthetic image
  obj_list, obj_ids = sample_objects(anno_list, categories_dict, min_obj, max_obj)

  # Create an image to combine instances
  comb_label = np.zeros((synt_height,synt_width), dtype=np.uint8) 
  background = sample_background(img_bg_path)
  comb_image = cv2.resize(background, (synt_width, synt_height))
  
  for obj_id, obj in zip(obj_ids, obj_list):
    print('obj_id ', obj_id, ' - ', categories_dict[obj_id], ' - ', images_dict[obj['image_id']])
    obj_img = cv2.imread(join(individual_objects_path, images_dict[obj['image_id']]))
  
    comb_image, comb_label, mask = add_obj_to_img(obj_img, 
                                                  obj_id, 
                                                  comb_image, 
                                                  comb_label)
    
  
  annos, ann_id = add_annotations(annos, ann_id, image_id, obj_ids, obj_list, comb_label, mask)


  # Add image to the list of images
  im_file_name = f'{image_id:04d}.jpg'
  im_dict = get_image_dict(im_file_name, synt_width, synt_height)
  images.append(im_dict)
  # save image
  cv2.imwrite(join(outdir, im_file_name), comb_image)
  cv2_imshow(comb_image)

# Add images to the main node data
data['images'] = images

# Add annotations to the main node data 
data['annotations'] = annos # get_annotations(obj_ids, obj_list, comb_label, mask, image_id)

# Add categories to the main node data
data['categories'] = get_categories(categories_dict)

# write JSON
json_path = join(outdir, "annotations.json")
with open(json_path, 'w') as outfile:
    json.dump(data, outfile)
